In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import PathPatch
from matplotlib.path import Path

# 设置样式
plt.style.use('default')
mpl.rcParams['font.family'] = 'Arial'
mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.linewidth'] = 1.0

# 加载数据
yearly_avg = pd.read_csv('yearly_climate_shock_averages.csv')

# 创建图表
fig, ax = plt.subplots(figsize=(10, 6), dpi=300)

# 定义颜色
colors = ['#D99BBB', '#9EC4BE', '#7DAEE0', '#E9C46A']
variables = ['tmmn_pos_num', 'tmmn_neg_num', 'pr_pos_num', 'pr_neg_num']
#labels = ['Positive temperature\nextremes', 'Negative temperature\nextremes', 
          #'Positive precipitation\nextremes', 'Negative precipitation\nextremes']

labels = ['Extreme heat', 'Extreme cold', 
          'Excessive rainfall', 'Rainfall deficits']

# 获取x值和宽度
x = yearly_avg['Year'].values
width = 0.5

# 创建堆叠柱状图
bottoms = np.zeros(len(x))
tops = []

for i, (var, label) in enumerate(zip(variables, labels)):
    heights = yearly_avg[var].values

    # 绘制柱状图
    ax.bar(x, heights, bottom=bottoms, width=width, color=colors[i], label=label,
           edgecolor='black', linewidth=0.5, zorder=10)

    # 存储每个堆叠的顶部位置
    tops.append(bottoms + heights)
    bottoms += heights

# 创建流图效果
def create_flow(x1, y1_bottom, y1_top, x2, y2_bottom, y2_top, knot_pos=0.5):
    """创建流图路径"""
    # 控制点位置
    cx = x1 + (x2 - x1) * knot_pos
    
    # 创建贝塞尔曲线路径
    verts = [
        (x1, y1_bottom),  # 起点底部
        (cx, y1_bottom),  # 控制点底部
        (x2, y2_bottom),  # 终点底部
        (x2, y2_top),     # 终点顶部
        (cx, y1_top),     # 控制点顶部
        (x1, y1_top),     # 起点顶部
        (x1, y1_bottom),  # 回到起点
    ]
    
    codes = [
        Path.MOVETO,      # 移动到起点
        Path.CURVE3,      # 二次贝塞尔曲线
        Path.CURVE3,      # 二次贝塞尔曲线的终点
        Path.LINETO,      # 直线到终点顶部
        Path.CURVE3,      # 二次贝塞尔曲线
        Path.CURVE3,      # 二次贝塞尔曲线的终点
        Path.CLOSEPOLY,   # 闭合路径
    ]
    
    return Path(verts, codes)

# 绘制流图
for i in range(len(variables)):
    for j in range(len(x) - 1):
        # 获取当前和下一个柱子的底部和顶部位置
        if i == 0:
            bottom1, bottom2 = 0, 0
        else:
            bottom1, bottom2 = tops[i-1][j], tops[i-1][j+1]
            
        top1, top2 = tops[i][j], tops[i][j+1]
        
        # 创建流图路径
        path = create_flow(
            x[j], bottom1, top1, 
            x[j+1], bottom2, top2,
            knot_pos=0.5  # 控制曲率
        )
        
        # 创建稍微浅色的颜色
        base_color = np.array(mpl.colors.to_rgb(colors[i]))
        lighter_color = base_color + (1 - base_color) * 0.6
        
        # 添加流图
        patch = PathPatch(
            path, facecolor=lighter_color, edgecolor='white',
            linewidth=1.5, alpha=0.7, zorder=6
        )
        ax.add_patch(patch)

# 自定义图表
ax.set_xlabel('Year', fontweight='bold')
ax.set_ylabel('Number of weather extremes', fontweight='bold')
#ax.set_title('Annual Climate Shocks (1990-2023)', fontweight='bold', fontsize=14)

# 移除网格线
ax.grid(False)

# 添加黑色边框
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_color('black')
    spine.set_linewidth(0.8)

# 设置y轴从0开始
ax.set_ylim(bottom=0)

# 添加图例
legend = ax.legend(frameon=True, loc='upper center', bbox_to_anchor=(0.5, -0.12),
                  ncol=4, framealpha=0.9, edgecolor='white')
legend.get_frame().set_linewidth(0.5)

# 调整布局
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# 保存图表
plt.savefig('climate_shocks_flow.png', dpi=500, bbox_inches='tight')

plt.show()